In [61]:
import wurst
import bw2io, bw2data
import bw2calc
import pandas as pd
import numpy as np
import matplotlib as plt

In [3]:
bw2io.restore_project_directory("/srv/data/ecoinvent-3.9.1-cutoff.tar.gz", project_name="K_iss")

Restoring project backup archive - this could take a few minutes...


'K_iss'

In [5]:
bw2data.projects.set_current("K_iss")

In [6]:
bw2data.databases

Databases dictionary with 2 object(s):
	biosphere3
	ecoinvent-3.9.1-cutoff

In [42]:
eidb = bw2data.Database("ecoinvent-3.9.1-cutoff")

In [85]:
df = pd.read_csv('FR_2022_hourly.csv', usecols=["Datetime (UTC)", "CO2"])
df

,Datetime (UTC),CO2
0,01/01/2022 00:00,0.04384
1,01/01/2022 01:00,0.04622
2,01/01/2022 02:00,0.04636
3,01/01/2022 03:00,0.04506
4,01/01/2022 04:00,0.04469
...,...,...
8755,31/12/2022 19:00,0.03908
8756,31/12/2022 20:00,0.03870
8757,31/12/2022 21:00,0.03999
8758,31/12/2022 22:00,0.03931


In [86]:
df.sort_values(by='Datetime (UTC)')

,Datetime (UTC),CO2
0,01/01/2022 00:00,0.04384
1,01/01/2022 01:00,0.04622
2,01/01/2022 02:00,0.04636
3,01/01/2022 03:00,0.04506
4,01/01/2022 04:00,0.04469
...,...,...
8755,31/12/2022 19:00,0.03908
8756,31/12/2022 20:00,0.03870
8757,31/12/2022 21:00,0.03999
8758,31/12/2022 22:00,0.03931


In [88]:
from statistics import mean

year= 2022
timestep = pd.Series(pd.date_range(start=f'{year}-01-01', end=f'{year}-12-31 23:00:00', freq='H'))
dates = pd.date_range(start=f'{year}-01-01', end=f'{year}-12-31')
num_repeats = 24

a= pd.DataFrame({'Timestamp': timestep})
a['Date']= pd.DataFrame(dates.repeat(num_repeats),columns=['date'])
a['Hour'] = pd.to_datetime(a['Timestamp']).dt.strftime('%H:%M:%S')
a['CO2']=df["CO2"]

In [89]:
a

,Timestamp,Date,Hour,CO2
0,2022-01-01 00:00:00,2022-01-01,00:00:00,0.04384
1,2022-01-01 01:00:00,2022-01-01,01:00:00,0.04622
2,2022-01-01 02:00:00,2022-01-01,02:00:00,0.04636
3,2022-01-01 03:00:00,2022-01-01,03:00:00,0.04506
4,2022-01-01 04:00:00,2022-01-01,04:00:00,0.04469
...,...,...,...,...
8755,2022-12-31 19:00:00,2022-12-31,19:00:00,0.03908
8756,2022-12-31 20:00:00,2022-12-31,20:00:00,0.03870
8757,2022-12-31 21:00:00,2022-12-31,21:00:00,0.03999
8758,2022-12-31 22:00:00,2022-12-31,22:00:00,0.03931


In [90]:
def sort_data(group):
    return group.sort_values('CO2', ascending=True)

In [91]:
grouped_data = a.groupby('Date').apply(sort_data)
grouped_data

Timestamp       Date      Hour      CO2
Date                                                             
2022-01-01 13   2022-01-01 13:00:00 2022-01-01  13:00:00  0.03661
           17   2022-01-01 17:00:00 2022-01-01  17:00:00  0.03696
           14   2022-01-01 14:00:00 2022-01-01  14:00:00  0.03706
           16   2022-01-01 16:00:00 2022-01-01  16:00:00  0.03707
           15   2022-01-01 15:00:00 2022-01-01  15:00:00  0.03749
...                             ...        ...       ...      ...
2022-12-31 8753 2022-12-31 17:00:00 2022-12-31  17:00:00  0.04000
           8739 2022-12-31 03:00:00 2022-12-31  03:00:00  0.04031
           8736 2022-12-31 00:00:00 2022-12-31  00:00:00  0.04088
           8738 2022-12-31 02:00:00 2022-12-31  02:00:00  0.04157
           8759 2022-12-31 23:00:00 2022-12-31  23:00:00  0.04217

[8760 rows x 4 columns]

In [92]:
# charge= pd.read_csv('emobpy_timeseries_hourly.csv')#, usecols=["datetime", "Consumer_0_elec"])

ele_cons_day=18.58 #kWh of EV ID 80 of the 01/01/2020

In [93]:
opt= grouped_data.iloc[::24, :]
opt

,,Timestamp,Date,Hour,CO2
Date,,,,,
2022-01-01,13,2022-01-01 13:00:00,2022-01-01,13:00:00,0.03661
2022-01-02,40,2022-01-02 16:00:00,2022-01-02,16:00:00,0.03353
2022-01-03,48,2022-01-03 00:00:00,2022-01-03,00:00:00,0.03898
2022-01-04,74,2022-01-04 02:00:00,2022-01-04,02:00:00,0.03391
2022-01-05,98,2022-01-05 02:00:00,2022-01-05,02:00:00,0.04184
...,...,...,...,...,...
2022-12-27,8663,2022-12-27 23:00:00,2022-12-27,23:00:00,0.03536
2022-12-28,8676,2022-12-28 12:00:00,2022-12-28,12:00:00,0.03161
2022-12-29,8688,2022-12-29 00:00:00,2022-12-29,00:00:00,0.03679


In [94]:
opt_con=opt['CO2'].multiply(ele_cons_day)
opt_con.sum()

502.902073

In [95]:
pes= grouped_data.iloc[::23, :]
pes

Timestamp       Date      Hour      CO2
Date                                                             
2022-01-01 13   2022-01-01 13:00:00 2022-01-01  13:00:00  0.03661
           2    2022-01-01 02:00:00 2022-01-01  02:00:00  0.04636
2022-01-02 28   2022-01-02 04:00:00 2022-01-02  04:00:00  0.04363
2022-01-03 67   2022-01-03 19:00:00 2022-01-03  19:00:00  0.05152
2022-01-04 88   2022-01-04 16:00:00 2022-01-04  16:00:00  0.06744
...                             ...        ...       ...      ...
2022-12-27 8657 2022-12-27 17:00:00 2022-12-27  17:00:00  0.04227
2022-12-28 8667 2022-12-28 03:00:00 2022-12-28  03:00:00  0.03328
2022-12-29 8698 2022-12-29 10:00:00 2022-12-29  10:00:00  0.03899
2022-12-30 8722 2022-12-30 10:00:00 2022-12-30  10:00:00  0.03722
2022-12-31 8749 2022-12-31 13:00:00 2022-12-31  13:00:00  0.03835

[381 rows x 4 columns]

In [97]:
pes_con=pes['CO2'].multiply(ele_cons_day)
pes_con.sum()

633.7104754

In [98]:
avg_fr= [act for act in eidb if 'market for electricity, low voltage'in act['name']
                    and 'FR' in act['location']][0]

avg_fr

'market for electricity, low voltage' (kilowatt hour, FR, None)

In [99]:
ipcc=[method for method in bw2data.methods if 'IPCC' in str(method) 
                                #and 'Endpoint' not in str(method)
                                and 'climate change' in str(method)
                                #and 'GWP100a' in str(method)
                                #and 'IPCC 2021' in str(method)
     ][0]
print(ipcc)
print(type(ipcc))

('IPCC 2013 no LT', 'climate change no LT', 'global temperature change potential (GTP100) no LT')
<class 'tuple'>


In [100]:
lca = bw2calc.LCA({avg_fr:1}, ipcc)
lca.lci()
lca.lcia()
lca.score

0.08141798883602024

In [101]:
avg_con=lca.score*ele_cons_day*365
avg_con

552.1523748892384